In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [ ]:


## for visualizing 
import matplotlib.pyplot as plt, numpy as np
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/drive/')
print(os.getcwd())

In [ ]:
import string
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
data=open("metamorphosis_clean.txt","r",encoding="utf8")


In [ ]:

lines=[]
for l in data:
   lines.append(l)

In [ ]:
strings=""
strings=" ".join(lines)

In [ ]:
strings=strings.replace('\n',"").replace('\r',"").replace('\ufeff',"")

In [ ]:
translator=str.maketrans(string.punctuation,' '*len(string.punctuation))
data=strings.translate(translator)

In [ ]:
len(data)

119163

In [ ]:
z=[]
for i in strings.split():
   if i not in z:
     z.append(i)
new_data=' '.join(z)     

In [ ]:
len(new_data)

29361

In [ ]:
tokenizer=Tokenizer()# tkenizer takes all the words individually and then it xonverts each word into particular number for each word .and new text will reptresent by new number
tokenizer.fit_on_texts([new_data])
pickle.dump(tokenizer, open('tokenizer1.pkl', 'wb'))

import pickle
sequence_data=tokenizer.texts_to_sequences([new_data])[0]
sequence_data[:10]
vocab_size=len(tokenizer.word_index)+1
print(len(sequence_data))
print(vocab_size)

3890
2617


In [ ]:
#we represents the sentences i to the combination of two words.
# sequence data is our data represented by number. converts to combination odf two words.
combination=[]
for i in range(1,len(sequence_data)):
  combination.append(sequence_data[i-1:i+1])
comb_array=np.array(combination)
print(len(comb_array))  

3889


In [ ]:
D=[]
T=[]

for i in comb_array:
  D.append(i[0])
  T.append(i[1])
D=np.array(D)
T=np.array(T) 
T=to_categorical(T)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size,10, input_length=1))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1, 10)             26170     
_________________________________________________________________
lstm (LSTM)                  (None, 1, 1000)           4044000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 1000)              8004000   
_________________________________________________________________
dense (Dense)                (None, 1000)              1001000   
_________________________________________________________________
dense_1 (Dense)              (None, 2617)              2619617   
Total params: 15,694,787
Trainable params: 15,694,787
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

checkpoint = ModelCheckpoint("nextword1.h5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto')

reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)

logdir='logsnextword1'
tensorboard_Visualization = TensorBoard(log_dir=logdir)

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=.001))
model.fit(D,T,epochs=150,batch_size=64,callbacks=[checkpoint,reduce,tensorboard_Visualization])

Epoch 1/150
 1/61 [..............................] - ETA: 0s - loss: 7.8698WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
60/61 [============================>.] - ETA: 0s - loss: 7.8751
Epoch 00001: loss improved from inf to 7.87523, saving model to nextword1.h5
61/61 [==============================] - 2s 36ms/step - loss: 7.8752
Epoch 2/150
61/61 [==============================] - ETA: 0s - loss: 7.8631
Epoch 00002: loss improved from 7.87523 to 7.86310, saving model to nextword1.h5
61/61 [==============================] - 2s 35ms/step - loss: 7.8631
Epoch 3/150
61/61 [==============================] - ETA: 0s - loss: 7.8182
Epoch 00003: loss improved from 7.86310 to 7.81823, saving model to nextword1.h5
61/61 [==============================] - 2s 34ms/step 

In [ ]:
# Importing the Libraries

from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer

model = load_model('/content/nextword1.h5')
tokenizer = pickle.load(open('/content/tokenizer1.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):
    """
        In this function we are using the tokenizer and models trained
        and we are creating the sequence of the text entered and then
        using our model to predict and return the the predicted word.
    
    """
    for i in range(3):
        sequence = tokenizer.texts_to_sequences([text])[0]
        sequence = np.array(sequence)
        
        preds = model.predict_classes(sequence)
#         print(preds)
        predicted_word = ""
        
        for key, value in tokenizer.word_index.items():
            if value == preds:
                predicted_word = key
                break
        
        print(predicted_word)
        return predicted_word

In [ ]:
"""
    We are testing our model and we will run the model
    until the user decides to stop the script.
    While the script is running we try and check if 
    the prediction can be made on the text. If no
    prediction can be made we just continue.

"""

# text1 = "at the dull"
# text2 = "collection of textile"
# text3 = "what a strenuous"
# text4 = "stop the script"

while(True):

    text = input("Enter your line: ")
    
    if text == "stop the script":
        print("Ending The Program.....")
        break
    
    else:
        try:
            text = text.split(" ")
            text = text[-1]

            text = ''.join(text)
            Predict_Next_Words(model, tokenizer, text)
            
        except:
            continue

Enter your line: how are
Enter your line: at the dull


KeyboardInterrupt: ignored